In [ ]:
import datetime
from pathlib import Path
import csv
import asyncio

import xlwings as xw

import stock
import stock.marketspeed.constants as mc

In [ ]:
excel_path = stock.constants.PROJECT_ROOT / "data/rss_data_collection_workbook.xlsx"
code_list_csv = stock.constants.PROJECT_ROOT / "data/margin_trade_target_list.csv"
output_dir = Path(r"G:\マイドライブ\stock\data")

wb = xw.Book(excel_path)
sheet = wb.sheets[0]

with open(code_list_csv, "r", encoding="utf-8") as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    codes = [row[0] for row in csv_reader]

In [ ]:
async def collect_data(sheet: xw.Sheet, code: str, output_dir: Path, count: int = 5 * 60 + 2):
    sheet.range("A1").value = f"=RssChart(,\"{code}.T\",\"1M\",{count})"
    await asyncio.sleep(1)
    values = sheet[f"D3:J{3 + count - 1}"].options(ndim=2).value

    timestamp = values[0][0].replace("/", "")
    output_path = output_dir / f"{timestamp}_{code}.csv"
    with open(output_path, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["day", "time", "open", "high", "low", "close", "volume"])
        writer.writerows(values)

    return output_path

In [ ]:
for code in codes:
    output_path = await collect_data(sheet, code, output_dir)
    print(output_path)